# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.

### 1. Import libraries and load data from database.
* Import Python libraries
* Load dataset from database with read_sql_table
* Define feature and target variables X and Y

In [1]:
!pip install nltk
!pip install plotly

import nltk

nltk.download(['punkt','stopwords','wordnet'])
nltk.download('averaged_perceptron_tagger')

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sys
import os
import re
import pickle
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection  import GridSearchCV
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.metrics import precision_recall_fscore_support,accuracy_score,label_ranking_average_precision_score


[nltk_data] Downloading package punkt to /Users/tina 1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/tina
[nltk_data]     1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tina 1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tina 1/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
def load_data(db_path='../data/disaster.db',tablename='_table'):
    """
    Function: load data from database and return X and y.
    Args:
      db_path(str): database file name included path
      tablename:(str): table name in the database file.
    Return:
      X(pd.DataFrame): messages for X
      y(pd.DataFrame): labels part in messages for y
    """
    
    # load data from database
    engine = create_engine('sqlite:///'+db_path)
    table_name = os.path.basename(db_path).replace(".db","") + "_table"
    df = pd.read_sql_table(table_name,engine)
    
    #Remove child alone as it has all zeros only
    df = df.drop(['child_alone'],axis=1)
    df['related']=df['related'].map(lambda x: 1 if x == 2 else x)
    X = df['message']
    y = df.iloc[:,4:]
    return X, y

In [3]:
X,y=load_data()
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
    """
    Tokenize the text function
    
    Arguments:
        text -> Text message which needs to be tokenized
    Output:
        clean_tokens -> List of tokens extracted from the provided text
    """
    
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

### 3. Build a machine learning pipeline

In [5]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
            ('tfidf_transformer', TfidfTransformer())
        ]))
            
    ])),

    ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
])
    

### 4. Train pipeline

In [6]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('count_vectorizer',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f9b855b8bf8>)),
                                                                 ('tfidf_transformer',
                                                                  TfidfTransformer())]))])),
                ('classifier',
                 MultiOutputClassifier(estimator=AdaBoostClassifier()))])

### 5. Test your model

In [7]:
def evaluate_model(y_test, y_pred):
    result=precision_recall_fscore_support(y_test, y_pred)
    for i, col in enumerate(y_test.columns.values): #category_names
        accu=accuracy_score(y_test.loc[:,col],y_pred[:,i])
        print('{}\n Accuracy: {:.4f}    % Precision: {:.4f}   % Recall {:.4f} '.format(
            col,accu,result[0][i],result[1][i]))
    avg_precision = label_ranking_average_precision_score(y_test, y_pred)
    avg_score= ('label ranking average precision: {}'.format(avg_precision))
    print(avg_score)

In [8]:
print("Evaluation model")

y_prediction_train = pipeline.predict(X_train)
y_prediction_test = pipeline.predict(X_test)

evaluate_model(y_train, y_prediction_train)

Evaluation model
related
 Accuracy: 0.8077    % Precision: 0.8313   % Recall 0.9403 
request
 Accuracy: 0.8987    % Precision: 0.7938   % Recall 0.5501 
offer
 Accuracy: 0.9953    % Precision: 0.4000   % Recall 0.0851 
aid_related
 Accuracy: 0.7620    % Precision: 0.7736   % Recall 0.6022 
medical_help
 Accuracy: 0.9323    % Precision: 0.6624   % Recall 0.2844 
medical_products
 Accuracy: 0.9619    % Precision: 0.7233   % Recall 0.3665 
search_and_rescue
 Accuracy: 0.9756    % Precision: 0.6995   % Recall 0.2296 
security
 Accuracy: 0.9817    % Precision: 0.4384   % Recall 0.0853 
military
 Accuracy: 0.9734    % Precision: 0.6809   % Recall 0.3860 
water
 Accuracy: 0.9674    % Precision: 0.7819   % Recall 0.6784 
food
 Accuracy: 0.9476    % Precision: 0.8199   % Recall 0.6794 
shelter
 Accuracy: 0.9486    % Precision: 0.7975   % Recall 0.5569 
clothing
 Accuracy: 0.9892    % Precision: 0.7583   % Recall 0.4776 
money
 Accuracy: 0.9806    % Precision: 0.6616   % Recall 0.3537 
missing_p

In [9]:
evaluate_model(y_test, y_prediction_test)

related
 Accuracy: 0.8018    % Precision: 0.8280   % Recall 0.9342 
request
 Accuracy: 0.8972    % Precision: 0.7953   % Recall 0.5320 
offer
 Accuracy: 0.9949    % Precision: 0.0000   % Recall 0.0000 
aid_related
 Accuracy: 0.7549    % Precision: 0.7682   % Recall 0.5818 
medical_help
 Accuracy: 0.9252    % Precision: 0.6138   % Recall 0.2667 
medical_products
 Accuracy: 0.9525    % Precision: 0.6027   % Recall 0.3154 
search_and_rescue
 Accuracy: 0.9760    % Precision: 0.6316   % Recall 0.1765 
security
 Accuracy: 0.9796    % Precision: 0.1333   % Recall 0.0208 
military
 Accuracy: 0.9720    % Precision: 0.5775   % Recall 0.2595 
water
 Accuracy: 0.9599    % Precision: 0.7148   % Recall 0.6209 
food
 Accuracy: 0.9521    % Precision: 0.8491   % Recall 0.6935 
shelter
 Accuracy: 0.9460    % Precision: 0.7697   % Recall 0.5641 
clothing
 Accuracy: 0.9907    % Precision: 0.7838   % Recall 0.4143 
money
 Accuracy: 0.9798    % Precision: 0.5556   % Recall 0.2679 
missing_people
 Accuracy: 

### 6. Improve your model
Use grid search to find better parameters.

In [10]:
parameters = {'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
              'classifier__estimator__n_estimators': [10, 20, 40]}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3, scoring='f1_weighted', verbose=3)

cv.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=10, score=0.466, total=  22.8s
[CV] classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.8s remaining:    0.0s


[CV]  classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=10, score=0.455, total=  22.8s
[CV] classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   45.6s remaining:    0.0s


[CV]  classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=10, score=0.464, total=  22.9s
[CV] classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=20 
[CV]  classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=20, score=0.452, total=  37.6s
[CV] classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=20 
[CV]  classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=20, score=0.453, total=  39.9s
[CV] classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=20 
[CV]  classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=20, score=0.450, total=  38.7s
[CV] classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=40 
[CV]  classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=40, score=0.425, total= 1.1min
[CV] classifier__estimator__learning_rate=0.01, classifier__estimato

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 19.6min finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7f9b855b8bf8>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'classifier__estimator__learning_rate': [0.01, 0.02,
                                                                  0.05],
                         'classifier__estimato

In [11]:
# Best parameters set
cv.best_params_

{'classifier__estimator__learning_rate': 0.01,
 'classifier__estimator__n_estimators': 10}

### 7. Test your model¶
Show the accuracy, precision, and recall of the tuned model.

Since this project focuses on code quality, process, and pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
evaluate_model(y_train, y_prediction_train)

related
 Accuracy: 0.8077    % Precision: 0.8313   % Recall 0.9403 
request
 Accuracy: 0.8987    % Precision: 0.7938   % Recall 0.5501 
offer
 Accuracy: 0.9953    % Precision: 0.4000   % Recall 0.0851 
aid_related
 Accuracy: 0.7620    % Precision: 0.7736   % Recall 0.6022 
medical_help
 Accuracy: 0.9323    % Precision: 0.6624   % Recall 0.2844 
medical_products
 Accuracy: 0.9619    % Precision: 0.7233   % Recall 0.3665 
search_and_rescue
 Accuracy: 0.9756    % Precision: 0.6995   % Recall 0.2296 
security
 Accuracy: 0.9817    % Precision: 0.4384   % Recall 0.0853 
military
 Accuracy: 0.9734    % Precision: 0.6809   % Recall 0.3860 
water
 Accuracy: 0.9674    % Precision: 0.7819   % Recall 0.6784 
food
 Accuracy: 0.9476    % Precision: 0.8199   % Recall 0.6794 
shelter
 Accuracy: 0.9486    % Precision: 0.7975   % Recall 0.5569 
clothing
 Accuracy: 0.9892    % Precision: 0.7583   % Recall 0.4776 
money
 Accuracy: 0.9806    % Precision: 0.6616   % Recall 0.3537 
missing_people
 Accuracy: 

In [13]:
evaluate_model(y_test, y_prediction_test)

related
 Accuracy: 0.8018    % Precision: 0.8280   % Recall 0.9342 
request
 Accuracy: 0.8972    % Precision: 0.7953   % Recall 0.5320 
offer
 Accuracy: 0.9949    % Precision: 0.0000   % Recall 0.0000 
aid_related
 Accuracy: 0.7549    % Precision: 0.7682   % Recall 0.5818 
medical_help
 Accuracy: 0.9252    % Precision: 0.6138   % Recall 0.2667 
medical_products
 Accuracy: 0.9525    % Precision: 0.6027   % Recall 0.3154 
search_and_rescue
 Accuracy: 0.9760    % Precision: 0.6316   % Recall 0.1765 
security
 Accuracy: 0.9796    % Precision: 0.1333   % Recall 0.0208 
military
 Accuracy: 0.9720    % Precision: 0.5775   % Recall 0.2595 
water
 Accuracy: 0.9599    % Precision: 0.7148   % Recall 0.6209 
food
 Accuracy: 0.9521    % Precision: 0.8491   % Recall 0.6935 
shelter
 Accuracy: 0.9460    % Precision: 0.7697   % Recall 0.5641 
clothing
 Accuracy: 0.9907    % Precision: 0.7838   % Recall 0.4143 
money
 Accuracy: 0.9798    % Precision: 0.5556   % Recall 0.2679 
missing_people
 Accuracy: 

### 8. Try improving your model further. Here are a few ideas:
1. try other machine learning algorithms
2. add other features besides the TF-IDF

In [14]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])


cv2 = GridSearchCV(pipeline2, param_grid=parameters, scoring='f1_micro', n_jobs=-1)

cv2.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7f9b855b8bf8>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb_transformer',
                                                                        StartingVerbExtractor())])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jo

### 9. Export your model as a pickle file

In [15]:
def save_model(cv):
    """
    Function: save model as pickle file.
    Args:
      cv:target model
    Return:
      N/A
    """
    
    with open('classifier.pkl', 'wb') as file:
        pickle.dump(cv, file)
    

In [16]:
print("Save model")
save_model(cv2)

Save model


### 10. Use this notebook to complete train.py
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.